In [1]:
from pyspark.sql import SparkSession

In [32]:
from pyspark.sql.functions import corr

In [2]:
from pyspark.sql.functions import split, regexp_replace, col, when, sum

In [3]:
spark = SparkSession.builder.appName("ReadHDFSFile").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/15 15:29:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/15 15:29:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [125]:
# competitions = spark.read.csv("hdfs://localhost:9000/big-data/competitions.csv",header=True)
events = spark.read.csv("Data/events.csv",header=True)
#frames = spark.read.csv("Data/frames.csv",header=True)
# lineups = spark.read.csv("hdfs://localhost:9000/big-data/lineups.csv",header=True)
matches = spark.read.csv("Data/matches.csv",header=True)

In [ ]:
for i in events.orderBy('id').limit(5).collect():
    print(i)

Row(50_50=None, bad_behaviour_card=None, ball_receipt_outcome=None, ball_recovery_offensive=None, ball_recovery_recovery_failure=None, block_deflection=None, block_offensive=None, carry_end_location=None, clearance_aerial_won=None, clearance_body_part=None, clearance_head=None, clearance_left_foot='0.0', clearance_right_foot=None, counterpress=None, dribble_nutmeg=None, dribble_outcome=None, dribble_overrun=None, duel_outcome=None, duel_type=None, duration=None, foul_committed_advantage=None, foul_committed_card=None, foul_committed_type='1b29d6b4-99d8-4cf0-9349-87690fa04702', foul_won_advantage='4', foul_won_defensive=None, goalkeeper_body_part=None, goalkeeper_end_location='7572', goalkeeper_outcome='0', goalkeeper_position=None, goalkeeper_technique=None, goalkeeper_type=None, id=None, index=None, injury_stoppage_in_chain=None, interception_outcome=None, location=None, match_id=None, minute=None, off_camera=None, out=None, pass_aerial_won=None, pass_angle=None, pass_assisted_shot_id

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)


In [5]:
frames.show()

+--------------------+--------------------+--------+--------+-----+------+--------------------+
|                  id|        visible_area|match_id|teammate|actor|keeper|            location|
+--------------------+--------------------+--------+--------+-----+------+--------------------+
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False|  True|[22.6975416639254...|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|[46.5466540256494...|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|[47.4452205797475...|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|[48.8195457108986...|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|[49.7459543709158...|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|[53.4689340941401...|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|[59.3929593689272...|
|25dfc952-7a85-464...|[82.0588035227948.

### Splitting Location

In [6]:
fs = frames.first()
fs['location']

'[22.697541663925428, 41.01277368603656]'

In [7]:
frames = frames.withColumn("location_clean", regexp_replace(col("location"), r"[\[\]]", "")) \
               .withColumn("x", split(col("location_clean"), ", ").getItem(0).cast("double")) \
               .withColumn("y", split(col("location_clean"), ", ").getItem(1).cast("double")) \
               .drop("location_clean").drop("location")

frames.show(22)

+--------------------+--------------------+--------+--------+-----+------+------------------+------------------+
|                  id|        visible_area|match_id|teammate|actor|keeper|                 x|                 y|
+--------------------+--------------------+--------+--------+-----+------+------------------+------------------+
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False|  True|22.697541663925428| 41.01277368603656|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False| 46.54665402564941| 46.65992069066397|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False| 47.44522057974756| 64.52929870627902|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False| 48.81954571089864|35.810976155696935|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False| 49.74595437091587|27.859841158806844|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|    True|False| False|53.468934094140124|48.

### Replacing True/False with 1/0

In [8]:
frames.printSchema()

root
 |-- id: string (nullable = true)
 |-- visible_area: string (nullable = true)
 |-- match_id: string (nullable = true)
 |-- teammate: string (nullable = true)
 |-- actor: string (nullable = true)
 |-- keeper: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)



In [9]:
columns_re = ['teammate','actor','keeper']

for column in columns_re:
    frames = frames.withColumn(column,when(col(column)=='True',1) \
    .when(col(column) == 'False',0).otherwise(col(column)))

frames.show(5)

+--------------------+--------------------+--------+--------+-----+------+------------------+------------------+
|                  id|        visible_area|match_id|teammate|actor|keeper|                 x|                 y|
+--------------------+--------------------+--------+--------+-----+------+------------------+------------------+
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|       1|    0|     1|22.697541663925428| 41.01277368603656|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|       1|    0|     0| 46.54665402564941| 46.65992069066397|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|       1|    0|     0| 47.44522057974756| 64.52929870627902|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|       1|    0|     0| 48.81954571089864|35.810976155696935|
|25dfc952-7a85-464...|[82.0588035227948...| 3942819|       1|    0|     0| 49.74595437091587|27.859841158806844|
+--------------------+--------------------+--------+--------+-----+------+------------------+---

In [10]:
# According to documentation, visible_area data is in format
# X1, Y1, X2, Y2 ... Xn, Yn

In [11]:
null_counts = frames.select(
    *[
        sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
        for c in frames.columns
    ]
)
null_counts.show()

+---+------------+--------+--------+-----+------+---+---+
| id|visible_area|match_id|teammate|actor|keeper|  x|  y|
+---+------------+--------+--------+-----+------+---+---+
|  0|           0|       0|       0|    0|     0|  0|  0|
+---+------------+--------+--------+-----+------+---+---+



In [22]:
events_s = events.select('id','shot_statsbomb_xg')
frames_s = frames.filter(frames.actor==1).select('id','x','y')

In [23]:
result = events_s.join(frames_s,"id","inner")

In [27]:
result.na.drop().count()

4844

In [50]:
a = events_s.na.drop(subset='shot_statsbomb_xg')

In [51]:
a.show()

+--------------------+-----------------+
|                  id|shot_statsbomb_xg|
+--------------------+-----------------+
|3c3fd192-a115-47b...|       0.07213958|
|b164d43a-c2cd-423...|       0.02977089|
|417a2357-d337-43c...|       0.07589752|
|30702c96-5580-48d...|       0.15686217|
|970398e3-c3cc-4d2...|       0.15133068|
|95ea28e2-8fe9-426...|      0.049562205|
|37ba4e86-d809-40e...|      0.015171255|
|f80a63b3-c4a6-4a9...|       0.08510399|
|5cf2e059-1c44-409...|       0.02531394|
|c9353ed3-a087-4fd...|     0.0064736516|
|f4ea5b80-1de5-444...|      0.047531724|
|8592fda4-3e54-46e...|      0.022013947|
|51932a82-e803-4ba...|       0.06569337|
|a8b4baa8-b879-410...|      0.012626733|
|a431db47-7da0-4eb...|       0.14169177|
|350a49bc-6914-4b3...|      0.040732585|
|e5d66514-0a8c-446...|      0.028598836|
|618314a1-76b9-41e...|      0.019570593|
|9b52e5c5-5688-4b0...|      0.056397837|
|2593d42f-61d2-4e5...|      0.039566465|
+--------------------+-----------------+
only showing top

In [ ]:
events

In [66]:
events.filter(events.type=='Shot').show()

+-----+------------------+--------------------+-----------------------+------------------------------+----------------+---------------+------------------+--------------------+-------------------+--------------+-------------------+--------------------+------------+--------------+---------------+---------------+------------+---------+--------+------------------------+-------------------+-------------------+------------------+------------------+--------------------+-----------------------+------------------+-------------------+--------------------+---------------+--------------------+-----+------------------------+--------------------+-------------+--------+------+----------+----+---------------+----------+---------------------+--------------+----------+-------------+-----------------+----------------+-----------+---------------+-----------+---------------------+------------+----------------+--------------+-----------------+----------------+-----------+--------------+-----------------+---

In [52]:
result = a.join(frames_s,"id","left")

In [54]:
result.count()

31905

In [124]:
for i in result.limit(5).collect():
    print(i)

+--------------------+-----------------+----+----+
|                  id|shot_statsbomb_xg|   x|   y|
+--------------------+-----------------+----+----+
|163515c5-cfe1-4b8...|        0.0806289|NULL|NULL|
|3c3fd192-a115-47b...|       0.07213958|NULL|NULL|
|477e37a4-c42e-4db...|            Off T|NULL|NULL|
|71f6b79d-2346-432...|            Off T|NULL|NULL|
|9cefc622-75a2-451...|       0.12105144|NULL|NULL|
+--------------------+-----------------+----+----+



Row(id='163515c5-cfe1-4b8c-9f06-1506f21b1253', shot_statsbomb_xg='0.0806289', x=None, y=None)
Row(id='3c3fd192-a115-47bc-9224-b5011bb17885', shot_statsbomb_xg='0.07213958', x=None, y=None)
Row(id='477e37a4-c42e-4db1-ab80-95829a60f6c9', shot_statsbomb_xg='Off T', x=None, y=None)
Row(id='71f6b79d-2346-4329-8a62-161146b712a8', shot_statsbomb_xg='Off T', x=None, y=None)
Row(id='9cefc622-75a2-4512-a91e-2f20044e12bb', shot_statsbomb_xg='0.12105144', x=None, y=None)


In [121]:
result.show()

+--------------------+--------------------+-----------------+------------------+
|                  id|   shot_statsbomb_xg|                x|                 y|
+--------------------+--------------------+-----------------+------------------+
|1495792e-cef5-4b4...| {'location': [100.9|95.30000305175781|26.299999237060547|
|163515c5-cfe1-4b8...|           0.0806289|             NULL|              NULL|
|20cb6277-e527-4e5...|               Saved|             NULL|              NULL|
|2b4e7844-74f7-43d...|         0.009314878|             NULL|              NULL|
|3c3fd192-a115-47b...|          0.07213958|             NULL|              NULL|
|477e37a4-c42e-4db...|               Off T|             NULL|              NULL|
|6926b425-5a2d-484...|         0.055555455|            100.0| 46.79999923706055|
|71f6b79d-2346-432...|               Off T|             NULL|              NULL|
|7da0626f-5c31-424...|             Wayward|             NULL|              NULL|
|851a137d-7441-4e3...|      

In [78]:
cor = result.na.drop().drop('id')

In [35]:
cor.show()

+-----------------+------------------+------------------+
|shot_statsbomb_xg|                 x|                 y|
+-----------------+------------------+------------------+
|       0.15243307|             114.0|40.900001525878906|
|      0.025271386| 93.80000305175781| 51.29999923706055|
|        0.6657846| 111.4000015258789|              36.5|
|       0.06717054| 108.9000015258789|29.399999618530273|
|      0.029664174|110.19999694824219|32.599998474121094|
|      0.030132728|              93.0|49.400001525878906|
|       0.05725451|             105.5|47.400001525878906|
|        0.0673188|111.19999694824219|              37.0|
|      0.069653474| 112.0999984741211|34.599998474121094|
|        0.8521293| 115.5999984741211| 43.79999923706055|
|      0.043656476|103.30000305175781|53.099998474121094|
|       0.02594082|102.19999694824219| 43.20000076293945|
|       0.08625654| 110.5999984741211|45.400001525878906|
|           0.7835|             108.0|              40.0|
|      0.02172

In [46]:
cor_df = cor.toPandas()
cor_df

,shot_statsbomb_xg,x,y
0,0.152433,114.000000,40.900002
1,0.025271,93.800003,51.299999
2,0.665785,111.400002,36.500000
3,0.067171,108.900002,29.400000
4,0.029664,110.199997,32.599998
...,...,...,...
4839,0.058865,104.300003,26.299999
4840,NaN,97.900002,29.200001
4841,0.045205,96.500000,40.900002
4842,0.086119,112.400002,29.500000


In [48]:
cor_df.sort_values(by='shot_statsbomb_xg',ascending=False)

,shot_statsbomb_xg,x,y
84,0.963011,119.000000,40.900002
1278,0.940326,116.699997,38.599998
2775,0.931034,115.900002,43.000000
3378,0.930487,108.599998,37.700001
3897,0.911103,114.599998,41.700001
...,...,...,...
4789,NaN,89.500000,30.200001
4796,NaN,107.599998,42.200001
4808,NaN,108.099998,40.500000
4826,NaN,111.300003,38.400002


In [34]:
cor.printSchema()

root
 |-- shot_statsbomb_xg: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)



In [40]:
from pyspark.sql.types import DoubleType

cor = cor.withColumn("shot_statsbomb_xg",cor['shot_statsbomb_xg'].cast(DoubleType()))

In [41]:
columns = cor.columns
correlations = []
for i in range(len(columns)):
    row = []
    for j in range(len(columns)):
        corr_val = cor.stat.corr(columns[i],columns[j])
        row.append(corr_val)
    correlations.append(row)

In [45]:
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt

corr_df = pd.DataFrame(correlations,index=columns,columns=columns)
corr_df

,shot_statsbomb_xg,x,y
shot_statsbomb_xg,1.000000,0.382066,0.012606
x,0.382066,1.000000,0.010299
y,0.012606,0.010299,1.000000


In [113]:
evs = events.select('id','shot_statsbomb_xg','location')
evs.count()

8710714

In [120]:
events.na.drop(subset='shot_statsbomb_xg').count()

31905

In [109]:
evs = evs.na.drop(subset='shot_statsbomb_xg')

In [114]:
evs.count()

8710714

In [117]:
evs.orderBy('location').show()

+----+-----------------+--------+
|  id|shot_statsbomb_xg|location|
+----+-----------------+--------+
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
|NULL|             NULL|    NULL|
+----+-----------------+--------+
only showing top 20 rows



In [92]:
evs = evs.na.drop(subset='location')
evs.count()

25536

In [111]:
evs = evs.withColumn("location_clean", regexp_replace(col("location"), r"[\[\]]", "")) \
               .withColumn("x", split(col("location_clean"), ", ").getItem(0).cast("double")) \
               .withColumn("y", split(col("location_clean"), ", ").getItem(1).cast("double")) \
               .drop("location_clean").drop("location")
               
evs.show()

+--------------------+-----------------+-----+----+
|                  id|shot_statsbomb_xg|    x|   y|
+--------------------+-----------------+-----+----+
|3c3fd192-a115-47b...|       0.07213958|105.6|44.0|
|b164d43a-c2cd-423...|       0.02977089| 98.6|25.2|
|417a2357-d337-43c...|       0.07589752|105.5|47.3|
|30702c96-5580-48d...|       0.15686217|113.4|38.7|
|970398e3-c3cc-4d2...|       0.15133068|106.2|36.8|
|95ea28e2-8fe9-426...|      0.049562205|108.3|39.0|
|37ba4e86-d809-40e...|      0.015171255| 85.6|34.4|
|f80a63b3-c4a6-4a9...|       0.08510399|112.5|42.5|
|5cf2e059-1c44-409...|       0.02531394|106.1|32.5|
|c9353ed3-a087-4fd...|     0.0064736516| 83.5|48.0|
|f4ea5b80-1de5-444...|      0.047531724|108.1|39.3|
|8592fda4-3e54-46e...|      0.022013947|105.5|57.6|
|51932a82-e803-4ba...|       0.06569337|110.9|25.6|
|a8b4baa8-b879-410...|      0.012626733| 98.9|24.9|
|a431db47-7da0-4eb...|       0.14169177|114.8|44.1|
|350a49bc-6914-4b3...|      0.040732585|107.2|35.0|
|e5d66514-0a

In [112]:
evs.count()

31905

In [98]:
evs = evs.drop('id')

In [107]:
evs.orderBy('y').show(50)

+--------------------+---------+----+
|   shot_statsbomb_xg|        x|   y|
+--------------------+---------+----+
|             Blocked|3825739.0|NULL|
|               Saved|3901207.0|NULL|
|               Off T|3900548.0|NULL|
|               Off T|3901207.0|NULL|
|  'teammate': False}|3900527.0|NULL|
|                Post|3901207.0|NULL|
|               Off T|3900548.0|NULL|
|                Goal|3901207.0|NULL|
|               Saved| 267422.0|NULL|
|               Off T|3901207.0|NULL|
|             Blocked|3900548.0|NULL|
|             Blocked|3901207.0|NULL|
|  'teammate': False}|3900527.0|NULL|
|             Blocked|3901207.0|NULL|
|               Off T|3900548.0|NULL|
|               Off T|3901207.0|NULL|
|   'teammate': True}|3825679.0|NULL|
|               Saved|3901207.0|NULL|
|               Off T|3900548.0|NULL|
|               Saved|3901207.0|NULL|
|                Goal|3900527.0|NULL|
|             Blocked|3901207.0|NULL|
|             Blocked|3900548.0|NULL|
|           

In [12]:
#spark.stop()